# Results Analysis

Load past experiments, compare configurations, generate reports

## Load Database

In [ ]:
from ris_research_engine.engine import ResultAnalyzer
from ris_research_engine.foundation import ResultTracker

tracker = ResultTracker("outputs/experiments/results.db")
analyzer = ResultAnalyzer(tracker)

## Show All Experiments

In [ ]:
experiments = tracker.get_all_experiments()
print(f"Total experiments: {len(experiments)}")

import pandas as pd

if experiments:
    df = pd.DataFrame([{
        'id': e['id'],
        'name': e['name'][:40],
        'probe': e['probe_type'],
        'model': e['model_type'],
        'M': e['M'],
        'K': e['K'],
        'accuracy': e['metrics'].get('top_1_accuracy', 0),
        'power_ratio': e['metrics'].get('power_ratio', 0),
        'status': e['status'],
        'time': f"{e['training_time_seconds']:.1f}s"
    } for e in experiments])
    display(df)
else:
    print("No experiments found. Run some experiments first!")

## Probe Comparison

In [ ]:
probe_comparison = analyzer.compare_probes()

if not probe_comparison.empty:
    print("Probe Performance Comparison:")
    print("="*60)
    display(probe_comparison)
    
    # Plot
    fig = analyzer.plot_probe_comparison()
    if fig:
        display(fig)
else:
    print("Not enough data for probe comparison")

## Model Comparison

In [ ]:
model_comparison = analyzer.compare_models()

if not model_comparison.empty:
    print("Model Performance Comparison:")
    print("="*60)
    display(model_comparison)
    
    # Plot
    fig = analyzer.plot_model_comparison()
    if fig:
        display(fig)
else:
    print("Not enough data for model comparison")

## Sparsity Analysis

In [ ]:
sparsity_analysis = analyzer.sparsity_analysis()

if not sparsity_analysis.empty:
    print("Sparsity Analysis:")
    print("="*60)
    display(sparsity_analysis.head(10))
    
    # Plot accuracy vs M/K
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    for probe in sparsity_analysis['probe_type'].unique():
        data = sparsity_analysis[sparsity_analysis['probe_type'] == probe]
        plt.plot(data['M']/data['K'], data['top_1_accuracy'], marker='o', label=probe)
    
    plt.xlabel('Sensing Budget (M/K)')
    plt.ylabel('Top-1 Accuracy')
    plt.legend()
    plt.title('Accuracy vs Sparsity')
    plt.grid(True)
    plt.show()
else:
    print("Not enough data for sparsity analysis")

## Fidelity Gap Analysis

In [ ]:
gap_analysis = analyzer.fidelity_gap_analysis()

if not gap_analysis.empty:
    print("Cross-Fidelity Gap Analysis:")
    print("="*60)
    display(gap_analysis)
    
    # Plot
    fig = analyzer.plot_fidelity_gap()
    if fig:
        display(fig)
else:
    print("No cross-fidelity experiments found")
    print("Run experiments with different data_fidelity values to enable this analysis")

## Export Best Configuration

In [ ]:
best = analyzer.best_configuration(metric='top_1_accuracy')

if best:
    print("Best Configuration:")
    print("="*60)
    print(f"Name: {best.config.name}")
    print(f"Probe: {best.config.probe_type}")
    print(f"Model: {best.config.model_type}")
    print(f"M/K: {best.config.system.M}/{best.config.system.K}")
    print(f"Accuracy: {best.metrics['top_1_accuracy']:.3f}")
    print(f"Training time: {best.training_time_seconds:.1f}s")
    
    # Export to JSON
    import json
    output_file = 'outputs/best_config.json'
    with open(output_file, 'w') as f:
        json.dump(best.config.to_dict(), f, indent=2)
    print(f"\n✅ Saved to {output_file}")
else:
    print("No completed experiments found")

## View Training Curves

In [ ]:
# Get the best experiment's training history
if best:
    # Find the experiment ID from the database
    best_exp = tracker.get_best_experiment(metric_name='top_1_accuracy')
    if best_exp:
        fig = analyzer.plot_training_curves(best_exp['id'])
        if fig:
            display(fig)
else:
    print("No experiments to display")